In [ ]:
import numpy as np
from numpy.linalg import norm as Enorm # Норма евклида 


# Положения источников 
A1 = np.array((0, -500, 0))
B1 = np.array((100, -500, 0))
A2 = np.array((0, 0, 0))
B2 = np.array((100, 0, 0))
A3 = np.array((0, 500, 0))
B3 = np.array((100, 500, 0))

# Положения приемников 
M1 = np.array((200, 0, 0))
N1 = np.array((300, 0, 0))
M2 = np.array((500, 0, 0))
N2 = np.array((600, 0, 0))
M3 = np.array((1000, 0, 0))
N3 = np.array((1100, 0, 0))

A = [A1, A2, A3]
B = [B1, B2, B3]
M = [M1, M2, M3]
N = [N1, N2, N3]



I = np.array((0,1,0)) # Истенное значение силы тока каждого источника. Все остальные равны нулю, т.к по заданию только источник под номером 2. 
delta_sigma = 0.0 # Смещение для поиска параметра проводимости
sigma_approx = 0.1 # Начальное приближение для sigma
sigma_n = sigma_approx # На n шаге 
sigma_true = 1.1 # Проводимость среды истенная 

alpha = 0.0 # Параметр регуляризации 

a11 = 0.0 # Параметр системы уравнений 
b1 = 0.0 # Правая часть 

# Потенциал на измерителе с учетом того, что источников тока 3 штуки 
# A,B - массив координат источника 
# M, N - точка измерителя 
# I - массив токов источников 
# sigma - коэффициент проводимости 
def V_AB_MN(A, B, M, N, I, sigma):

    res = 0.0

    for i in range(0, 3):
        const_val = I[i]/(2*np.pi * sigma)
        r_BM = Enorm(B[i]-M)
        r_AM = Enorm(A[i]-M)
        r_BN = Enorm(B[i]-N)
        r_AN = Enorm(A[i]-N)

        val2 = 1.0/r_BM - 1.0/r_AM
        val3 = 1.0/r_BN - 1.0/r_AN

        res = res + const_val*(val2 - val3) 
    
    return res # Значение напряжения на линии 


# Производная напряжения по параметру сигма 
# A,B - массив координат источника 
# M, N - точка измерителя 
# I - массив токов источников 
# sigma - коэффициент проводимости, истенный или приближенный. 
def dV_AB_MN_dsigma_I(A, B, M, N, I, sigma):
    res = 0.0
    
    const_val = 0.0

    for i in range(0, 3):
        
        const_val = -I[i]/(2*np.pi * (sigma**2)) 
    
        r_BM = Enorm(B[i]-M)
        r_AM = Enorm(A[i]-M)
        r_BN = Enorm(B[i]-N)
        r_AN = Enorm(A[i]-N)

        val2 = 1.0/r_BM - 1.0/r_AM
        val3 = 1.0/r_BN - 1.0/r_AN

        res = res + const_val*(val2 - val3) 

    return res # Значение напряжения на линии 



noise = [-10.0, 5.0, -10.0]
V = [V_AB_MN(A, B, Mi, Ni, I, sigma_true) for Mi, Ni in zip(M, N)] # Померенные значения напряжения 
# Шумим в измерения 
for i in range(0, 3):
    V[i] = V[i] + noise[i]*V[i]/100

# Веса 
w = np.array([1.0/V_AB_MN(A, B, M[0], N[0], I, sigma_n), 1.0/V_AB_MN(A, B, M[1], N[1], I, sigma_n), 1.0/V_AB_MN(A, B, M[2], N[2], I, sigma_n)])


def F(I, w, sigma_n, V):
    res = 0.0

    for i in range(0, 3):
        res = res + (w[i]*(V_AB_MN(A, B, M[i], N[i], I, sigma_n) - V[i]))**2
    return np.sqrt(res)

for iteration in range(0, 10):
    
    descripency = F(I, w, sigma_n, V)
    print("{n}\t{sigma:.7e}\t{F_sigma:.7e}\t".format(n = iteration, sigma=sigma_n, F_sigma = descripency))
    
    if descripency <= 1e-8 or iteration == 15:
        break
    

    for i in range(0, 3):
        w[i] = 1.0/V_AB_MN(A, B, M[i], N[i], I, sigma_n)
        a11 = a11 + (w[i]*dV_AB_MN_dsigma_I(A, B, M[i], N[i], I, sigma_n))**2

    for i in range(0, 3):
        
        r1 = w[i]**2
        r2 = dV_AB_MN_dsigma_I(A, B, M[i], N[i], I, sigma_n)
        r3 = (V_AB_MN(A, B, M[i], N[i], I, sigma_n)- V[i])
        b1 = b1 + r1*r2*r3
    
    sigma_n = sigma_n - b1 / a11



    a11 = 0.0
    b1 = 0.0


#print("sigma = {:.6e} delta = {:.2f}%".format(sigma_n,100.0*np.abs(sigma_true/sigma_n - 1)))



0	1.0000000e-01	1.5799100e+00	
1	1.9121212e-01	7.5376257e-01	
2	3.5029393e-01	6.5503097e-01	
3	5.9275546e-01	4.9313564e-01	
4	8.7674086e-01	2.8282343e-01	
5	1.0779799e+00	1.4982870e-01	
6	1.1347725e+00	1.6009549e-01	
7	1.1379223e+00	1.6846144e-01	
8	1.1379310e+00	1.6892903e-01	
9	1.1379310e+00	1.6893033e-01	
